In [21]:
from imports.TactileBag import TactileBag
import numpy as np

tbag = TactileBag('../data/bags/small4/')


In [22]:
tbag.params

{'path': '/home/hussain/me/projects/tactile/data/bags/small4',
 'possible_angles': [0.10471975511965978],
 'N_examples': 9,
 'theta': [0,
  0,
  0.22439948,
  0.44879895,
  0.67319843,
  0.8975979,
  1.12199738,
  1.34639685,
  1.57079633],
 'N_iters': 10,
 'z_thresh': -0.0037,
 'min_z': -0.015,
 'max_z': 0.035,
 'led': 1,
 'bg_filter': 80}

In [1]:
import sys

from imports.ExtractContactCases import ExtractContactCases
from imports.EventArrayAugmention import *
import math
import numpy as np
from pprint import pprint

ex = ExtractContactCases(
    tactile_bag_dir='../data/bags/small4/',
    outdir='../data/small/dense_quarter',
    n_init_events=10000,
)

pprint(ex.params)

{'N_examples': 9,
 'augmentations': [],
 'bag': '/home/hussain/me/projects/tactile/data/bags/small4',
 'case_span': 2660000000.0,
 'center': (180, 117),
 'circle_rad': 85,
 'delta_t': 75000000.0,
 'margin': -25000000.0,
 'min_n_events': 750,
 'n_init_events': 10000,
 'possible_angles': [0.10471975511965978],
 'theta': [0,
           0,
           0.22439948,
           0.44879895,
           0.67319843,
           0.8975979,
           1.12199738,
           1.34639685,
           1.57079633],
 'train_prop': 0.75}


In [2]:
ex.extract()

extracting contact timestamps:   0%|          | 0/16346 [00:00<?, ?it/s]

extracting event arrays:   0%|          | 0/80 [00:00<?, ?it/s]

saving
80 80


In [16]:
from models.modules import MaxPooling, MaxPoolingX
import torch
import torch_geometric

from torch.nn import Linear
from torch.nn.functional import elu
from torch_geometric.nn.conv import SplineConv
from torch_geometric.nn.norm import BatchNorm
from torch_geometric.transforms import Cartesian
from torch_geometric.data import Data, Batch
from torch_geometric.nn.pool import radius_graph, knn_graph

from models.modules import TransitionDown
from torch_geometric import seed_everything

seed_everything(0)

class GraphRes(torch.nn.Module):

    def __init__(self):
        super(GraphRes, self).__init__()
        dim = 3

        bias = False
        root_weight = False
        pooling_size=(16/346, 12/260)

        # Set dataset specific hyper-parameters.
        kernel_size = 2
        n = [1, 8, 16, 16, 16, 32, 32, 32, 32]
        pooling_outputs = 16
        #kernel_size = 8
        #n = [1, 16, 32, 32, 32, 128, 128, 128]
        #pooling_outputs = 128

        self.conv1 = SplineConv(n[0], n[1], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm1 = BatchNorm(in_channels=n[1])
        self.td1 = TransitionDown(8, 8, ratio=0.25)
        
        self.conv2 = SplineConv(n[1], n[2], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm2 = BatchNorm(in_channels=n[2])

        self.pool7 = MaxPoolingX(0.25, size=16)
        self.fc = Linear(pooling_outputs * 16, out_features=2, bias=bias)

    def forward(self, data: torch_geometric.data.Batch) -> torch.Tensor:
        data.x = elu(self.conv1(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm1(data.x)
        x, pos, batch = self.td1(data.x, data.pos, data.batch)
        
        edge_index = radius_graph(pos, r=0.05, max_num_neighbors=16)
        data = Batch(x=x, pos=pos, batch=batch, edge_index=edge_index, y=data.y)
        data = Cartesian()(data)
        data.x = elu(self.conv2(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm2(data.x)

        x = self.pool7(data.x, pos=data.pos[:, :2], batch=data.batch)
        x = x.view(-1, self.fc.in_features)
        return self.fc(x)

    
    
class GraphRes(torch.nn.Module):

    def __init__(self):
        super(GraphRes, self).__init__()
        dim = 3

        bias = False
        root_weight = False
        pooling_size=(16/346, 12/260)

        # Set dataset specific hyper-parameters.
        kernel_size = 2
        n = [1, 8, 16, 16, 16, 32, 32, 32, 32]
        pooling_outputs = 32
        #kernel_size = 8
        #n = [1, 16, 32, 32, 32, 128, 128, 128]
        #pooling_outputs = 128

        self.conv1 = SplineConv(n[0], n[1], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm1 = BatchNorm(in_channels=n[1])
        self.conv2 = SplineConv(n[1], n[2], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm2 = BatchNorm(in_channels=n[2])
        self.td2 = TransitionDown(n[2], n[2], 0.5)
        
        self.conv3 = SplineConv(n[2], n[3], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm3 = BatchNorm(in_channels=n[3])
        self.conv4 = SplineConv(n[3], n[4], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm4 = BatchNorm(in_channels=n[4])

        self.conv5 = SplineConv(n[4], n[5], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm5 = BatchNorm(in_channels=n[5])
        self.pool5 = MaxPooling(pooling_size, transform=Cartesian(norm=True, cat=False))

        self.conv6 = SplineConv(n[5], n[6], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm6 = BatchNorm(in_channels=n[6])
        self.conv7 = SplineConv(n[6], n[7], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm7 = BatchNorm(in_channels=n[7])

        self.pool7 = MaxPoolingX(0.25, size=16)
        self.fc = Linear(pooling_outputs * 16, out_features=2, bias=bias)

    def forward(self, data: torch_geometric.data.Batch) -> torch.Tensor:
        data.x = elu(self.conv1(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm1(data.x)
        data.x = elu(self.conv2(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm2(data.x)
        
        x, pos, batch = self.td2(data.x, data.pos, data.batch)
        edge_index = radius_graph(pos, r=0.05, max_num_neighbors=16)
        data = Batch(x=x, pos=pos, batch=batch, edge_index=edge_index, y=data.y)
        data = Cartesian()(data)
        
        x_sc = data.x.clone()
        data.x = elu(self.conv3(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm3(data.x)
        data.x = elu(self.conv4(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm4(data.x)
        data.x = data.x + x_sc

        data.x = elu(self.conv5(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm5(data.x)
        data = self.pool5(data.x, pos=data.pos, batch=data.batch, edge_index=data.edge_index, return_data_obj=True)

        x_sc = data.x.clone()
        data.x = elu(self.conv6(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm6(data.x)
        data.x = elu(self.conv7(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm7(data.x)
        data.x = data.x + x_sc

        x = self.pool7(data.x, pos=data.pos[:, :2], batch=data.batch)
        x = x.view(-1, self.fc.in_features)
        return self.fc(x)

In [17]:
model = GraphRes()
#model.load_state_dict(torch.load('../results/exp6/ckpt_20'))

In [18]:
from imports.TrainModel import TrainModel

#from models.modules import GraphRes

#!rm -rf ../data/small/dense_quarter/{test,train,val}/processed/*.pt
tm = TrainModel(
    '../data/small/dense_quarter/', 
    model.cuda(), 
    lr=0.001, 
    features='pol', 
    batch=4, 
    n_epochs=300, 
    experiment_name='exp6', 
    desc='aegnn model small on denser quarter',
    merge_test_val=True
)

{'bag': '/home/hussain/me/projects/tactile/data/bags/small4', 'train_prop': 0.75, 'delta_t': 75000000.0, 'margin': -25000000.0, 'min_n_events': 750, 'n_init_events': 10000, 'center': [180, 117], 'circle_rad': 85, 'case_span': 2660000000.0, 'N_examples': 9, 'possible_angles': [0.10471975511965978], 'theta': [0, 0, 0.22439948, 0.44879895, 0.67319843, 0.8975979, 1.12199738, 1.34639685, 1.57079633], 'augmentations': []}
{'bag': '/home/hussain/me/projects/tactile/data/bags/small4', 'train_prop': 0.75, 'delta_t': 75000000.0, 'margin': -25000000.0, 'min_n_events': 750, 'n_init_events': 10000, 'center': [180, 117], 'circle_rad': 85, 'case_span': 2660000000.0, 'N_examples': 9, 'possible_angles': [0.10471975511965978], 'theta': [0, 0, 0.22439948, 0.44879895, 0.67319843, 0.8975979, 1.12199738, 1.34639685, 1.57079633], 'augmentations': []}
{'bag': '/home/hussain/me/projects/tactile/data/bags/small4', 'train_prop': 0.75, 'delta_t': 75000000.0, 'margin': -25000000.0, 'min_n_events': 750, 'n_init_eve

In [19]:
tm.train()

training:   0%|          | 0/300 [00:00<?, ?epoch/s]

  0%|          | 0/15 [00:00<?, ?batch/s]

logging


  0%|          | 0/15 [00:00<?, ?batch/s]

logging


...

logging


  0%|          | 0/15 [00:00<?, ?batch/s]

logging


  0%|          | 0/15 [00:00<?, ?batch/s]

logging


In [10]:
loss = 0
losses = []
for i, data in enumerate(tm.val_loader):      
    data = data.to(tm.device)
    end_point = tm.model(data)
    l = tm.loss_func(end_point, data.y).detach().item()
    loss += l
    losses.append(l)
loss /= len(tm.val_loader)

0.045419190637767316